# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.core.environment import Environment 
from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azure.ai.ml import MLClient, load_component
import azure.ai.ml as aml

from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Environment
from azure.ai.ml import command, Input
from azure.ai.ml.constants import AssetTypes
import azure.ai.ml.sweep as sw

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)
print("azure-ai-ml versoins:", aml.__version__)

SDK version: 1.61.0
azure-ai-ml versoins: 1.31.0


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'sweep-heart-failure'
project_folder = './'

quick-starts-ws-295398
aml-quickstarts-295398
westeurope
3d1a56d2-7c81-4118-9790-f85d1acf0c77


In [3]:
experiment=Experiment(ws, experiment_name)
experiment

subscription_id="3d1a56d2-7c81-4118-9790-f85d1acf0c77"
resource_group="aml-quickstarts-295398"
workspace_name="quick-starts-ws-295398"

ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name
)

print("Connected to: ", ml_client.workspace_name)

Connected to:  quick-starts-ws-295398


In [4]:
data_asset = ml_client.data.get(name="hf-dataset", version="1")

print("name", data_asset.name)
print("version", data_asset.version)
print("type", data_asset.type)
print("path", data_asset.path)

name hf-dataset
version 1
type mltable
path azureml://subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-295398/workspaces/quick-starts-ws-295398/datastores/workspaceblobstore/paths/UI/2026-02-04_105703_UTC/heart_failure_clinical_records_dataset.csv/


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
# Choose a name for your CPU cluster
cluster_name = "compute00"

compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    
if compute_target:
    print('Found existing cluster, use it.')
else: 
    print("Not found under name " + cluster_name)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [12]:
env = Environment(
    name="hf-sweep-env",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    conda_file="conda_dependencies.yml"
)

ml_client.environments.create_or_update(env)

Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'hf-sweep-env', 'description': None, 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourceGroups/aml-quickstarts-295398/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-295398/environments/hf-sweep-env/versions/4', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook295398/code/Users/odl_user_295398', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x70db793ec790>, 'serialize': <msrest.serialization.Serializer object at 0x70dbc5311390>, 'version': '4', 'conda_file': {'dependencies': ['python=3.10', 'scikit-learn', 'numpy', 'pandas', {'pi

In [18]:
train_job = command(
    code=".",
    command=(
        "python train.py "
        "--training_data ${{inputs.training_data}} "
        "--label_col DEATH_EVENT "
        "--C ${{inputs.C}} "
        "--max_iter ${{inputs.max_iter}}"
        ),
    inputs={
        "training_data": Input(
            type=AssetTypes.MLTABLE,
            path="azureml:heart-failure-dataset:1"
        ),
        "C": 1.0,
        "max_iter": 100,
    },
    environment="hf-sweep-env:4",
    compute="compute00",
    experiment_name="sweep-heart-failure"
)

In [19]:
#Submit experiment
sweep_job = train_job.sweep(
    compute="compute00",
    sampling_algorithm="random",
    primary_metric="accuracy",
    goal="maximize",
)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [20]:
sweep_job.set_limits(max_total_trials=20, max_concurrent_trials=10, timeout=3600)

In [21]:
sweep_job.search_space = {
    "C":sw.LogUniform(min_value=1e-3, max_value=1e1),
    "max_iter":sw.Choice(values=[50, 100, 200, 500])
}

In [22]:
created = ml_client.jobs.create_or_update(sweep_job)
ml_client.jobs.stream(created.name)

Uploading odl_user_295398 (0.06 MBs): 100%|██████████| 62136/62136 [00:00<00:00, 758024.83it/s]




RunId: lucid_crayon_l314wnmgm0
Web View: https://ml.azure.com/runs/lucid_crayon_l314wnmgm0?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-295398/workspaces/quick-starts-ws-295398

Streaming azureml-logs/hyperdrive.txt

[2026-02-04T16:21:33.4560056Z][GENERATOR][DEBUG]Sampled 10 jobs from search space 
[2026-02-04T16:21:33.8880282Z][SCHEDULER][INFO]Scheduling job, id='lucid_crayon_l314wnmgm0_0' 
[2026-02-04T16:21:33.9966989Z][SCHEDULER][INFO]Scheduling job, id='lucid_crayon_l314wnmgm0_2' 
[2026-02-04T16:21:33.9504546Z][SCHEDULER][INFO]Scheduling job, id='lucid_crayon_l314wnmgm0_1' 
[2026-02-04T16:21:34.1415390Z][SCHEDULER][INFO]Scheduling job, id='lucid_crayon_l314wnmgm0_3' 
[2026-02-04T16:21:34.2347173Z][SCHEDULER][INFO]Scheduling job, id='lucid_crayon_l314wnmgm0_7' 
[2026-02-04T16:21:34.1436381Z][SCHEDULER][INFO]Scheduling job, id='lucid_crayon_l314wnmgm0_5' 
[2026-02-04T16:21:34.1447886Z][SCHEDULER][INFO]Scheduling job, id='lucid_crayon_l314wnm

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
best_run_file_names = best_run.get_file_names()
parameter_values = best_run.get_details()['runDefinition']['arguments']


print('Best Run Id: ', best_run.id)
print('Best Run Metrics: ', best_run_metrics) 
print('Best Run Accuracy:', best_run_metrics['Accuracy']) 
print('Best Run File Names:', best_run_file_names) 

# print the model parameter values for the best run
print('\n Model parameters: ')
for elem in parameter_values:
    print(elem)

In [ ]:
best_model_deploy = best_run.register_model(model_name='hf-best-model', model_path='outputs/model')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model = Model(ws, 'hf-best-model')
env = Environment.from_conda_specification(name='prediction-env', file_path='conda_dependencies.yml')

inference_config = InferenceConfig(entry_script='score.py', environment=env)

service_name = 'heart-failure-prediction'
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace = ws,
                       name = service_name,
                       models = [model],
                       inference_config = inference_config,
                       deployment_config = aci_config)

In [ ]:
service.wait_for_deployment(show_output=True)
print(service.scoring_uri)
print(service.state)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
auth = InteractiveLoginAuthentication()
header = auth.get_authentication_header()

data = {
  "data":
    {
      "age": 51, 
      "anaemia": 0, 
      "creatinine_phosphokinase": 600, 
      "diabetes": 1, 
      "ejection_fraction": 20, 
      "high_blood_pressure": 1, 
      "platelets": 265000, 
      "serum_creatinine": 1.9, 
      "serum_sodium": 130, 
      "sex": 0, 
      "smoking": 1,
      "time": 4
    },
  }

body = json.dumps(data)
response = requests.post(service.scoring_uri,
                         headers=header,
                         data=body)
print(response.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

